In [ ]:
import os

import cv2
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import medfilt

from skimage.filters.rank import entropy
from skimage.morphology import square

In [ ]:
def entropyBlk(filename, sz=3, threshold=0.9):
    image = cv2.imread(filename)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_out = entropy(grayscale, square(sz))
    
    # Rescale image to emphasize high-entropy regions
    theMax = np.amax(image_out)
    if theMax > 0:
        image_out /= theMax
    
    # Threshold the image
    image_out = np.where(image_out < threshold, 0, image_out)
    
    image_final = cv2.cvtColor(image_out.astype('float32'), cv2.COLOR_GRAY2RGB)
    
    return image_rgb,image_final

def entropyMask(filename, sz=11):
    # Wraps entropy function, using result as mask
    orig,entropy = entropyBlk(filename, sz, .8) # Hard code for now
    entropy_filt = medfilt(entropy, 3) # Hard code for now
    masked = orig * entropy_filt / 255 # Normalize
    
    final = cv2.cvtColor(masked.astype('float32'), cv2.COLOR_RGB2BGR)
    return orig,final
    

def procFiles(path, func, sz=15):
    filenames = [f for f in os.listdir(path) if f.endswith("PNG") or f.endswith("jpg")]
    os.makedirs(path + 'Processed/')
    for filename in filenames:
        print(path + filename)
        _,out = func(path + filename)
        name, ext = os.path.splitext(filename)
        full_path = path + 'Processed/' + name + '_entropy' + ext
        cv2.imwrite(full_path, 255 * out)



In [ ]:
if __name__ == '__main__':

    path = "../Images/Images_from_Liu_Bolin_s_site/"    
    procFiles(path, entropyMask)

#     plt.clf()
#     plt.figure(figsize=(sz, sz))
#     plt.imshow(final)
#     plt.figure(figsize=(sz, sz))
#     plt.imshow(orig)
#     plt.figure(figsize=(sz, sz))
#     plt.imshow(masked)